## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-07-18-51-56 +0000,nyt,Superpower Competition: The Missing Chapter in...,https://www.nytimes.com/2025/12/07/us/politics...
1,2025-12-07-18-49-00 +0000,wsj,Opinion | Britain Sidelines Juries to Silence ...,https://www.wsj.com/opinion/britain-sidelines-...
2,2025-12-07-18-47-21 +0000,nyt,The Impact of Trump’s Slipping Approval Rating,https://www.nytimes.com/video/us/politics/1000...
3,2025-12-07-18-44-42 +0000,nyt,"In Brazil’s Answer to Hollywood, Dreams and Dr...",https://www.nytimes.com/2025/12/07/world/ameri...
4,2025-12-07-18-44-16 +0000,startribune,"Live: J.J. McCarthy, Vikings lead the Commande...",https://www.startribune.com/minnesota-vikings-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2404/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,26
74,fire,13
285,new,10
119,will,7
157,deadly,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
152,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...,67
52,2025-12-07-13-42-20 +0000,cbc,"Israel's Netanyahu says he will meet Trump, se...",https://www.cbc.ca/news/world/netanyahu-says-h...,56
35,2025-12-07-16-24-42 +0000,nypost,Tim Walz’s daughter Hope goes on foul-mouthed ...,https://nypost.com/2025/12/07/us-news/tim-walz...,51
34,2025-12-07-16-27-00 +0000,wsj,President Trump criticized as disloyal an indi...,https://www.wsj.com/politics/elections/trump-l...,51
115,2025-12-07-05-41-09 +0000,nypost,Rosie O’Donnell’s therapist told her to ‘detac...,https://nypost.com/2025/12/07/media/rosie-odon...,50


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
152,67,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...
142,41,2025-12-06-23-09-00 +0000,wsj,Defense Secretary Pete Hegseth defended the U....,https://www.wsj.com/politics/national-security...
65,36,2025-12-07-12-33-07 +0000,nypost,Hochul will drive away New Yorkers if she supp...,https://nypost.com/2025/12/07/us-news/pataki-h...
107,31,2025-12-07-07-26-04 +0000,nypost,Florida teens lured 14-year-old girl into wood...,https://nypost.com/2025/12/07/us-news/florida-...
41,27,2025-12-07-15-27-55 +0000,nyt,Benin Coup Attempt to Oust Talon Has Been Foil...,https://www.nytimes.com/2025/12/07/world/afric...
147,27,2025-12-06-21-41-29 +0000,nypost,Christmas returns to Bethlehem with tree-light...,https://nypost.com/2025/12/06/world-news/chris...
25,25,2025-12-07-17-27-45 +0000,nypost,"Ilhan Omar compares Stephen Miller, who is Jew...",https://nypost.com/2025/12/07/us-news/ilhan-om...
170,25,2025-12-06-18-58-09 +0000,bbc,Can you ban kids from social media? Australia ...,https://www.bbc.com/news/articles/crk7xgzj8y8o...
103,24,2025-12-07-08-36-56 +0000,nypost,"At least 25 dead, including tourists, in fire ...",https://nypost.com/2025/12/07/world-news/india...
96,23,2025-12-07-10-00-00 +0000,wsj,With a deadline looming on Affordable Care Act...,https://www.wsj.com/politics/policy/aca-subsid...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
